In [1]:
cyr_chars = ['а', 'б', 'в', 'г', 'д', 'е', 'ё', 'ж', 'з', 'и', 'й',
             'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'х',
             'ф', 'ц', 'ч', 'щ', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я']

char2num = {}
num2char = {}

for i, ch in enumerate(cyr_chars):
    char2num[ch] = i + 1
    num2char[i + 1] = ch

char2num[' '] = 35
num2char[35] = ' '

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5269844224957405958
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 11585093078672622903
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 11572476197311652601
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4827840512
locality {
  bus_id: 1
  links {
  }
}
incarnation: 11421831612208046323
physical_device_desc: "device: 0, name: GeForce RTX 2060, pci bus id: 0000:02:00.0, compute capability: 7.5"
]


In [2]:
data_file_name = '/home/ilya/Documents/git/rus_lit/data_converters/converted'

with open(data_file_name, 'r') as f:
    data = f.readlines()

data = [s.strip() for s in data]

In [3]:
MIN_WORDS_COUNT = 6
word_sent = [s.split(' ') for s in data]
filtered_sent = [w for w in word_sent if len(w) >= MIN_WORDS_COUNT]
print(len(filtered_sent))
print(filtered_sent[:20])

5654
[['том', 'третий', 'государственное', 'издательство', 'художественной', 'литературы'], ['содержание', 'руслан', 'и', 'людмила', 'кавказский', 'пленник', 'гавриилиада', 'братьяразбойники', 'бахчисарайский', 'фонтан', 'цыганы', 'граф', 'нулин', 'полтава', 'тазит', 'домик', 'в', 'коломне', 'анджело', 'медный', 'всадник', 'завершнное', 'планы', 'отрывки', 'наброски', 'монах', 'бова', 'исповедь', 'поэма', 'о', 'гетеристах', 'иордаки', 'актеон', 'вадим', 'бова', 'мстислав', 'агасфер', 'кирджали', 'казачка', 'и', 'черкес', 'езерский', 'юдифь', 'из', 'ранних', 'редакции', 'с'], ['поэмы', 'пушкина', 'примечания', 'руслан', 'и', 'людмила', 'посвящение', 'для', 'вас', 'души', 'моей', 'царицы', 'красавицы', 'для', 'вас', 'одних', 'времен', 'минувших', 'небылицы', 'в', 'часы', 'досугов', 'золотых', 'под', 'шепот', 'старины', 'болтливой', 'рукою', 'верной', 'я', 'писал'], ['примите', 'ж', 'вы', 'мой', 'труд', 'игривый'], ['ничьих', 'не', 'требуя', 'похвал', 'счастлив', 'уж', 'я', 'надеждой', 'с

In [4]:
MAX_SENTENCE_LENGTH = 50

learning = list()
for word_sent in filtered_sent:
    sent = ''
    for w in word_sent:
        if len(sent + w) > MAX_SENTENCE_LENGTH:
            break
        sent = sent + w + ' '
    learning.append(sent.strip())

In [5]:
def convert2num(learning):
    nums_col = []
    for s in learning:
        nums_col.append([-1] + [char2num[w] for w in s])
    return nums_col

data_nums = convert2num(learning)

In [6]:
from keras.preprocessing.sequence import pad_sequences

ready_data = []
labels = []

for dd in data_nums:
    for i, _ in enumerate(dd):
        ready_data.append(pad_sequences([dd[:i + 1]], maxlen=MAX_SENTENCE_LENGTH + 1, padding='post')[0])
        labels.append(dd[i + 1] if i < len(dd) - 1 else -2 )

Using TensorFlow backend.


In [7]:
import numpy
import keras.utils as ku

ready_data = numpy.asarray(ready_data)
labels = numpy.asarray(labels)
labels = ku.to_categorical(labels, num_classes=len(char2num) + 2)

In [8]:
print(ready_data.shape)
print(labels.shape)

(257336, 51)
(257336, 36)


In [9]:
from keras.layers import Embedding, LSTM, Dense, Dropout, InputLayer
from keras.models import Sequential

def create_model(predictors, label, total_words):
    input_len = MAX_SENTENCE_LENGTH + 1
    model = Sequential()
    model.add(Embedding(total_words, 10, input_length=input_len))
    model.add(LSTM(300))
    model.add(Dropout(0.1))
    model.add(Dense(total_words, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    model.fit(predictors, label, epochs=30, verbose=1, validation_split=.1)
    
    return model

In [10]:
import tensorflow as tf

model = create_model(ready_data, labels, len(char2num) + 2)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 231602 samples, validate on 25734 samples
Epoch 1/30
231602/231602 [==============================] - 297s 1ms/step - loss: 2.9954 - val_loss: 2.6622
Epoch 2/30
231602/231602 [==============================] - 299s 1ms/step - loss: 2.5806 - val_loss: 2.4940
Epoch 3/30
231602/231602 [==============================] - 295s 1ms/step - loss: 2.4330 - val_loss: 2.3081
Epoch 4/30
231602/231602 [==============================] - 310s 1ms/step - loss: 2.2840 - val_loss: 2.2006
Epoch 5/30
231602/231602 [==============================] - 312s 1ms/step - loss: 2.1625 - val_loss: 2.0822
Epoch 6/30
231602/231602 [==============================] - 315s 1ms/step - loss: 2.0527 - val_loss: 2.0211
Epoch 7/30
231602/231602 [==============================] - 303s 1

In [13]:
def generate_text(seed_text, char_num, model):
    nex = len(seed_text) + 1
    tokinized = pad_sequences([[-1] + [char2num[ch] for ch in seed_text]], maxlen=MAX_SENTENCE_LENGTH + 1, padding='post')
    for i in range(char_num):
        next_char = model.predict_classes(tokinized)
        tokinized[0][nex] = next_char[0]
        nex = nex + 1
    print(''.join(num2char[ch] for ch in tokinized[0] if ch in num2char))

In [41]:
generate_text('пету', 30, model)

петушок объявил ее при ней просил 


In [11]:
model.save('/home/ilya/Documents/git/rus_lit/models/baseline/saved22.h5')